# Milestone 2: Data wrangling and initial analyses

## 1.  Familiarization with CMU Movie Summary Corpus dataset and restructuration

In this section, we familiarize ourselves with the CMU Movie Summary Corpus dataset. We also restructure the data to make it directly usable for our future analysis.

### 1.1 Character metadata

### 1.2 Movie metadata

### 1.3 Plot summaries

### 1.4 Core NLP plot summaries

## 2. Familiarization with IMDb dataset and restructuration

In this section, we familiarize ourselves with the IMDb dataset. We select the useful data for our future analysis.

### 2.1 Titles principals

### 2.2 Name basics

### 2.3 Title crew

### 2.4 Title ratings

### 2.5 Title basics

## 3. Data aggregation (IMDb and Movie Summary Corpus datasets)

In this section, we aggragate the IMDb and the Movie Summary Corpus together thanks to previously queried matches between IMDb IDs and freebase IDs.

### 3.1 Matching dataframe between IMDb IDs and freebase IDs

### 3.2 Merging the datasets

## 4. Data cleaning and crosschecking the data

In this section, we make sure that our dataset is clean and usable by making sanity checks and crosschecking data when possible.

# 5. Initial analysis

### 5.1 Are women younger in cinema ?

#### 5.1.1 Evolution through time

#### 5.2 Differences between genres

### 5.2 Description of characters in the summaries

### 5.3 Difference in importance of movie roles women and men get

### 5.4 Are IMDb ratings biased towards male/female opinion ?